<a href="https://colab.research.google.com/github/bryanjiang1204/aurora/blob/main/BERT_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
"""
Bert Embeddings
"""
import torch

input_text = "Wise Gurus chant"
input_ids = torch.tensor([tokenizer.encode(input_text)])
with torch.no_grad():
    model_output = model(input_ids)
bert_embeddings = model_output[0]

In [ ]:
from torchtext.datasets import IMDB

train_iter = IMDB(split='train')
test_iter = IMDB(split='test')

def tokenize(label, line):
    return line.split()

tokens = []
for label, line in train_iter:
    tokens += torch.tensor(tokenizer.encode(line))

In [ ]:
class MyModel(nn.Module):
    def __init__(self, embedding_dim, vocab_size):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embedding(x)  # x shape: (batch_size, seq_len, embedding_dim)
        x, _ = self.lstm(x)    # x shape: (batch_size, seq_len, hidden_size)
        x = x[:, -1, :]        # x shape: (batch_size, hidden_size)
        x = self.fc(x)         # x shape: (batch_size, num_classes)
        return x

embedding_dim = 768
vocab_size = len(tokenizer.vocab)
model = MyModel(embedding_dim, vocab_size)

# Set the embedding layer weights to the BERT embeddings
model.embedding.weight.data.copy_(bert_embeddings)
model.embedding.weight.requires_grad = False